In [4]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

# Load the dataset
df = pd.read_csv('/content/Base.csv')

# Calculate the amount for each class to make up 30% of the total dataframe
total_samples = int(0.6 * len(df))  # 60% of the main dataframe
samples_per_class = total_samples // 2  # 50% of 60% for each class

# Separate fraud and non-fraud instances
fraudulent = df[df['fraud_bool'] == 1]
non_fraudulent = df[df['fraud_bool'] == 0]

# Sample or oversample each class to get the required amount
fraudulent_sampled = fraudulent.sample(n=samples_per_class, replace=True, random_state=42)
non_fraudulent_sampled = non_fraudulent.sample(n=samples_per_class, replace=True, random_state=42)

# Concatenate the sampled data to create a new balanced dataframe
df_new = pd.concat([fraudulent_sampled, non_fraudulent_sampled])

# Split the new data into training and testing sets
X = df_new.drop(columns=['fraud_bool'])
y = df_new['fraud_bool']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# One-hot encode categorical variables
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# Ensure both training and test sets have the same columns
common_columns = list(set(X_train.columns) & set(X_test.columns))
X_train = X_train[common_columns]
X_test = X_test[common_columns]

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the deep learning model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, verbose=1)

# Train the model with early stopping
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Convert probabilities to class labels
y_pred_classes = (y_pred > 0.5).astype(int)

# Optionally, calculate additional evaluation metrics such as precision, recall, and F1-score
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_classes))


Epoch 1/10
9658/9658 [==============================] - 33s 3ms/step - loss: 0.4607 - accuracy: 0.7917 - val_loss: 0.4186 - val_accuracy: 0.8089
Epoch 2/10
9658/9658 [==============================] - 32s 3ms/step - loss: 0.4312 - accuracy: 0.8075 - val_loss: 0.4050 - val_accuracy: 0.8132
Epoch 3/10
9658/9658 [==============================] - 33s 3ms/step - loss: 0.4232 - accuracy: 0.8109 - val_loss: 0.3965 - val_accuracy: 0.8178
Epoch 4/10
9658/9658 [==============================] - 35s 4ms/step - loss: 0.4181 - accuracy: 0.8127 - val_loss: 0.3915 - val_accuracy: 0.8169
Epoch 5/10
9658/9658 [==============================] - 34s 3ms/step - loss: 0.4144 - accuracy: 0.8146 - val_loss: 0.3888 - val_accuracy: 0.8203
Epoch 6/10
9658/9658 [==============================] - 33s 3ms/step - loss: 0.4128 - accuracy: 0.8156 - val_loss: 0.3841 - val_accuracy: 0.8205
Epoch 7/10
9658/9658 [==============================] - 36s 4ms/step - loss: 0.4097 - accuracy: 0.8159 - val_loss: 0.3828 - val_ac

In [11]:
import pandas as pd

# Data from the classification report
data = {
    'Precision': [0.86, 0.80, 0.83, 0.83],
    'Recall': [0.78, 0.88, 0.83, 0.83],
    'F1-score': [0.82, 0.83, 0.83, 0.83],
    'Support': [38528, 38731, 77259, 77259]
}

# Index for the rows
index = ['0 (Non-Fraud)', '1 (Fraud)', 'Macro Avg', 'Weighted Avg']

# Create the DataFrame
df = pd.DataFrame(data, index=index)

# Display the DataFrame
df


,Precision,Recall,F1-score,Support
0 (Non-Fraud),0.86,0.78,0.82,38528
1 (Fraud),0.80,0.88,0.83,38731
Macro Avg,0.83,0.83,0.83,77259
Weighted Avg,0.83,0.83,0.83,77259


**The DataFrame provides a clear and structured view of the classification report**:

**Precision**: Indicates the ability of the classifier not to label as positive a sample that is negative. For example, out of all the instances predicted as fraud, 80% actually were fraud.

**Recall**: Also known as sensitivity, it indicates the ability to find all the positive samples. For example, the classifier correctly identifies 88% of all actual fraud cases.

**F1-score**: The weighted harmonic mean of precision and recall. An F1-score reaches its best value at 1 (perfect precision and recall) and worst at 0.

**Support**: The number of occurrences of each class in the actual true labels.

**Insights from the Classification Report**

**Balanced Performance**: The model has fairly balanced precision, recall, and F1-scores across both classes, which suggests that it performs equally well on both fraud and non-fraud predictions.

**Higher Recall for Fraud**: The recall for the fraud class (1) is higher than for the non-fraud class (0), indicating the model is slightly better at identifying fraudulent transactions compared to non-fraudulent ones.

**Precision vs. Recall**: The precision for non-fraud is higher than for fraud. This means the model is more conservative in predicting transactions as fraud, preferring to minimize false positives at the risk of missing some fraudulent cases (higher false negatives).

**Overall Accuracy**: The accuracy of 0.83 shows that the model correctly predicts the outcome of a transaction 83% of the time.